In [1]:
#Imports
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Load dataset and define maximum columns to be displayed so as to avoid some columns from being hidden
data = pd.read_json("S:/Xampp/htdocs/tweepy/tweets.json")
pd.set_option('display.max_columns', 500)

In [3]:
#Shuffle the data, to bring records with coordinates set to the top 10
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,limit,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,None,2019-02-26 07:44:27,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.100301e+18,1.100301e+18,None,NaN,NaN,NaN,NaN,0.0,tr,NaN,"{'id': '3078869807f9dd36', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,2019-02-26 07:44:27.135,0.0,"{'id': 993959236752338944, 'id_str': '99395923..."
1,NaN,None,2019-02-25 10:41:18,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.099983e+18,1.099983e+18,None,NaN,NaN,NaN,NaN,1.0,pt,NaN,"{'id': 'b994e5ab6edeabcc', 'url': 'https://api...",NaN,0.0,{'created_at': 'Mon Feb 25 08:55:02 +0000 2019...,1.099956e+18,1.099956e+18,"{'url': 'https://t.co/sBAlQeNm2M', 'expanded':...",0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,2019-02-25 10:41:18.352,0.0,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,NaN,None,2019-02-26 07:44:13,NaN,"{'hashtags': [{'text': 'Cancer', 'indices': [9...",NaN,NaN,0.0,0.0,low,None,1.100300e+18,1.100300e+18,None,NaN,NaN,NaN,NaN,1.0,en,NaN,"{'id': '7ae9e2f2ff7a87cd', 'url': 'https://api...",NaN,0.0,{'created_at': 'Mon Feb 25 13:36:16 +0000 2019...,1.100027e+18,1.100027e+18,"{'url': 'https://t.co/SS8dmzeSzF', 'expanded':...",0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,2019-02-26 07:44:13.446,0.0,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,NaN,None,2019-02-25 10:40:59,"[31, 31]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1099982574418415616, 'id_str...",NaN,0.0,0.0,low,None,1.099983e+18,1.099983e+18,parfaite_1223,1.099972e+18,1.099972e+18,7.031677e+17,7.031677e+17,0.0,und,NaN,"{'id': '019a3137395a2973', 'url': 'https://api...",0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,2019-02-25 10:40:59.530,0.0,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,NaN,None,2019-02-26 19:15:19,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.100474e+18,1.100474e+18,Mbuthuma,NaN,NaN,6.060205e+07,6.060205e+07,0.0,en,NaN,"{'id': '46c1b1ab24d7e11a', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,2019-02-26 19:15:19.983,0.0,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,NaN,None,2019-02-26 07:44:42,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.100301e+18,1.100301e+18,None,NaN,NaN,NaN,NaN,0.0,ar,NaN,"{'id': '001907e868d06e24', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,2019-02-26 07:44:42.070,0.0,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,NaN,None,2019-02-25 10:41:01,"[15, 21]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.099983e+18,1.099983e+18,greenhousenyt,1.099823e+18,1.099823e+18,2.684763e+08,2.684763e+08,0.0,und,NaN,"{'id': '5722ff20ba67083b', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,2019-02-25 10:41:01.017,0.0,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,NaN,None,2019-02-25 10:41:01,NaN,"{'hashtags': 

In [4]:
#Size of dataset
data.shape

(4147, 37)

In [5]:
#Find the null values
data.apply(lambda d: sum(d.isnull()), axis = 0)

contributors                 4147
coordinates                  3791
created_at                     33
display_text_range           2364
entities                       33
extended_entities            3762
extended_tweet               3451
favorite_count                 33
favorited                      33
filter_level                   33
geo                          3791
id                             33
id_str                         33
in_reply_to_screen_name      2619
in_reply_to_status_id        2707
in_reply_to_status_id_str    2707
in_reply_to_user_id          2619
in_reply_to_user_id_str      2619
is_quote_status                33
lang                           33
limit                        4114
place                         477
possibly_sensitive           3226
quote_count                    33
quoted_status                3543
quoted_status_id             3543
quoted_status_id_str         3543
quoted_status_permalink      3543
reply_count                    33
retweet_count 

In [6]:
#Drop columns that may not affect the analysis
data.drop(['id', 'created_at', 'contributors', 'display_text_range', 'extended_entities', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'limit', 'quote_count', 'quoted_status_id', 'quoted_status_id_str', 'favorited', 'filter_level', 'is_quote_status', 'quoted_status', 'quoted_status_permalink', 'timestamp_ms', 'truncated'], 1, inplace = True)
data.head(10)

,coordinates,entities,extended_tweet,favorite_count,geo,lang,place,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,tr,"{'id': '3078869807f9dd36', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,pt,"{'id': 'b994e5ab6edeabcc', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,None,"{'hashtags': [{'text': 'Cancer', 'indices': [9...",NaN,0.0,None,en,"{'id': '7ae9e2f2ff7a87cd', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,und,"{'id': '019a3137395a2973', 'url': 'https://api...",0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,en,"{'id': '46c1b1ab24d7e11a', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,ar,"{'id': '001907e868d06e24', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,und,"{'id': '5722ff20ba67083b', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,ja,"{'id': '24f07b6d792fa678', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,pt,"{'id': '97bcdfca1a2dca59', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,tl,"{'id': 'dd9c0d7d7e07eb49', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [7]:
#Find out what is in the entities column
freqs = data.entities.head(5)
for k, v in freqs.items():
    print(f'{k} {v}')

0 {'hashtags': [], 'urls': [], 'user_mentions': [], 'symbols': []}
1 {'hashtags': [], 'urls': [], 'user_mentions': [], 'symbols': []}
2 {'hashtags': [{'text': 'Cancer', 'indices': [9, 16]}], 'urls': [], 'user_mentions': [], 'symbols': []}
3 {'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'parfaite_1223', 'name': 'parfaite/パルフェット', 'id': 703167670003195906, 'id_str': '703167670003195906', 'indices': [0, 14]}, {'screen_name': 'parfaite_narumi', 'name': '鳴海ゆい🎀parfaite', 'id': 108347723, 'id_str': '108347723', 'indices': [15, 31]}], 'symbols': [], 'media': [{'id': 1099982574418415616, 'id_str': '1099982574418415616', 'indices': [32, 55], 'media_url': 'http://pbs.twimg.com/tweet_video_thumb/D0PsUs9UUAAzjKY.jpg', 'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/D0PsUs9UUAAzjKY.jpg', 'url': 'https://t.co/9rypI3ihNI', 'display_url': 'pic.twitter.com/9rypI3ihNI', 'expanded_url': 'https://twitter.com/LinXhiang/status/1099982584669323265/photo/1', 'type': 'photo', 'sizes

In [8]:
#Find out what is in the extended_tweet column
freqs = data.extended_tweet.head(5)
for k, v in freqs.items():
    print(f'{k} {v}')

0 nan
1 nan
2 nan
3 nan
4 nan


In [9]:
data.drop(['entities', 'extended_tweet'], 1, inplace = True)
data.head(10)

,coordinates,favorite_count,geo,lang,place,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,None,0.0,None,tr,"{'id': '3078869807f9dd36', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,None,0.0,None,pt,"{'id': 'b994e5ab6edeabcc', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,None,0.0,None,en,"{'id': '7ae9e2f2ff7a87cd', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,None,0.0,None,und,"{'id': '019a3137395a2973', 'url': 'https://api...",0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,None,0.0,None,en,"{'id': '46c1b1ab24d7e11a', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,None,0.0,None,ar,"{'id': '001907e868d06e24', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,None,0.0,None,und,"{'id': '5722ff20ba67083b', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,None,0.0,None,ja,"{'id': '24f07b6d792fa678', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,None,0.0,None,pt,"{'id': '97bcdfca1a2dca59', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,None,0.0,None,tl,"{'id': 'dd9c0d7d7e07eb49', 'url': 'https://api...",NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [10]:
#Drop place column because it indicates where the tweet is associated with but not necessarily originating from
data.drop(['place'], 1, inplace = True)
data.head(10)

,coordinates,favorite_count,geo,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,None,0.0,None,tr,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,None,0.0,None,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,None,0.0,None,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,None,0.0,None,und,0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,None,0.0,None,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,None,0.0,None,ar,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,None,0.0,None,und,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,None,0.0,None,ja,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,None,0.0,None,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,None,0.0,None,tl,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [11]:
#Find out what is in the coordinates column
freqs = data.coordinates.head(5)
for k, v in freqs.items():
    print(f'{k} {v}')

0 None
1 None
2 None
3 None
4 None


In [12]:
#Find out what is in the geo column
freqs = data.geo.head(5)
for k, v in freqs.items():
    print(f'{k} {v}')

0 None
1 None
2 None
3 None
4 None


In [13]:
#Drop geo as it contains what is in the coordinates
data.drop(['geo'], 1, inplace = True)
data.head(10)

,coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,None,0.0,tr,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,None,0.0,und,0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,None,0.0,ar,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,None,0.0,und,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,None,0.0,ja,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,None,0.0,tl,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [14]:
#Create a new coordinates column to hold the coordinatesit
data['geo_coordinates'] = data.coordinates

#Rename coordinates column to geo_tag to indicate whether the tweet has been geo-tagged
data.rename(columns={'coordinates':'geo_tag'}, inplace=True)
data.head(10)

,geo_tag,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user,geo_coordinates
0,None,0.0,tr,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923...",None
1,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name...",None
2,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':...",None
3,None,0.0,und,0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na...",None
4,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':...",None
5,None,0.0,ar,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name...",None
6,None,0.0,und,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name...",None
7,None,0.0,ja,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name...",None
8,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':...",None
9,None,0.0,tl,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name...",None


In [15]:
#Re-order columns
column_order = [0,11,1,2,3,4,5,6,7,8,9,10]
data = data[data.columns[column_order]]
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,None,None,0.0,tr,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,None,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,None,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,None,None,0.0,und,0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,None,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,None,None,0.0,ar,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,None,None,0.0,und,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,None,None,0.0,ja,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,None,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,None,None,0.0,tl,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [16]:
#Encode the geo_tag column
data['geo_tag'].fillna('False', inplace=True)
data.loc[data['geo_tag'] != 'False', 'geo_tag'] = 'True'
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,False,None,0.0,tr,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,False,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,False,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,False,None,0.0,und,0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,False,None,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,False,None,0.0,ar,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,False,None,0.0,und,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,False,None,0.0,ja,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,False,None,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,False,None,0.0,tl,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [17]:
#Obtain the coordinates key-value pair from the previous coordinates column
tags = data['geo_coordinates'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
tags

S:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
S:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
S:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,tag_type,tag_coordinates,tag_0
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [18]:
#Assign the new value for coordinates only to the column geo_coordinates created
data['geo_coordinates'] = tags.tag_coordinates
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,False,NaN,0.0,tr,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,False,NaN,0.0,und,0.0,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/android"" ...",@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,False,NaN,0.0,ar,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,False,NaN,0.0,und,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,False,NaN,0.0,ja,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,False,NaN,0.0,tl,NaN,0.0,0.0,0.0,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [19]:
#View the full source field
pd.set_option('display.max_colwidth', -1)
data.source

0       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
1       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
2       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
3       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
4       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
5       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
6       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
7       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
8       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
9       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
10      <a href="https://olplus.fr" rel="nofollow">OLPlusApp</a>      

In [20]:
data.loc[4118]

geo_tag               False                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [21]:
#A hypothesis at this point is that tweeting from an external app adds the geo-tag by default
#hence influences chances of a tweet being geo-tagged

#Therefore obtain meaningful information from the source column

#In the source column, the particular device/app is in between > and < i.e   (......> Device / App <......)
#I will use > and < as delimiters to obtain the device/app name alone

#Split the source column first based on >
first_split = data['source'].str.split(">", n = 1, expand = True)

#Obtain the second part of what was split and since its stored as an array the second is index 1
second_part = first_split[1]
second_part.head(10)

0    Twitter for Android</a>
1    Twitter for Android</a>
2    Twitter for iPhone</a> 
3    Twitter for Android</a>
4    Twitter for iPhone</a> 
5    Twitter for iPhone</a> 
6    Twitter for iPhone</a> 
7    Twitter for iPhone</a> 
8    Twitter for iPhone</a> 
9    Twitter for iPhone</a> 
Name: 1, dtype: object

In [22]:
#Split the variable holding the data above i.e. second_part

#Split using the < as the delimiter
last = second_part.str.split("<", n = 1, expand = True)

#Obtain the first part of the split which now contains the particular device/app alone and since its stored as an array the first is index 0
specific_names = last[0]
specific_names.head(10)

0    Twitter for Android
1    Twitter for Android
2    Twitter for iPhone 
3    Twitter for Android
4    Twitter for iPhone 
5    Twitter for iPhone 
6    Twitter for iPhone 
7    Twitter for iPhone 
8    Twitter for iPhone 
9    Twitter for iPhone 
Name: 0, dtype: object

In [23]:
#Assign the particular device/app names as the new source column values and rename it
data.source = specific_names

#Reset column length and print
pd.reset_option('display.max_colwidth')
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,False,NaN,0.0,tr,NaN,0.0,0.0,0.0,NaN,Twitter for Android,Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,Twitter for Android,Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,False,NaN,0.0,und,0.0,0.0,0.0,0.0,NaN,Twitter for Android,@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,False,NaN,0.0,ar,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,False,NaN,0.0,und,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,False,NaN,0.0,ja,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,False,NaN,0.0,tl,NaN,0.0,0.0,0.0,NaN,Twitter for iPhone,@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [24]:
#Further cleanup of the source column

#Assign every field with android, iphone or mobile field as mobile to show that it a mobile phone(handheld) 
data['source'] = data.source.str.replace(r'(^.*(Android|iPhone|Mobile).*$)', 'mobile')

data['source'] = data.source.str.replace(r'(^.*Web.*$)', 'web')
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user
0,False,NaN,0.0,tr,NaN,0.0,0.0,0.0,NaN,mobile,Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923..."
1,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,mobile,Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name..."
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,mobile,F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':..."
3,False,NaN,0.0,und,0.0,0.0,0.0,0.0,NaN,mobile,@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na..."
4,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,mobile,@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':..."
5,False,NaN,0.0,ar,NaN,0.0,0.0,0.0,NaN,mobile,جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name..."
6,False,NaN,0.0,und,NaN,0.0,0.0,0.0,NaN,mobile,@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name..."
7,False,NaN,0.0,ja,NaN,0.0,0.0,0.0,NaN,mobile,そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name..."
8,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,mobile,@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':..."
9,False,NaN,0.0,tl,NaN,0.0,0.0,0.0,NaN,mobile,@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name..."


In [25]:
#Find out the unique elements in source column
data.source.unique()

array(['mobile', 'OLPlusApp', 'Foursquare', 'IFTTT', 'web',
       'Twitter for iPad', 'CareerArc 2.0', nan, 'Instagram',
       'Tweetlogix', 'Trendsmap Alerting', 'Hootsuite Inc.',
       'Foursquare Swarm', 'Tweetbot for iΟS', 'Echofon',
       'dasauge® Stellenmarkt', 'iOS', 'Endomondo', 'manager magazin',
       'TTN ELP traffic', 'PropertyWala.com', '511NY-Tweets',
       '112alarm Lifeliners', 'Echobox Social', 'Cheap Bots, Done Quick!',
       'LA AreaNotice Tweets', 'TweetDeck', 'srvnotif', 'NYRegions',
       '112alarm.net MON', 'twitbot_h', 'SocialFlow', 'Facebook',
       'GroupTweet', 'swonkie', '112app.nl', 'Tuiteo TOP EP (4)',
       'Twitterrific for Mac', '112alarm.net', 'SafeTweet by TweetMyJOBS',
       'Mobolet Tweet', 'iembot', 'TestsForSearch', 'TTN SPO traffic',
       "Edent's Solar Panels", 'Tweetbot for Mac',
       'City of Portland Fire/EMS feed', 'eltiempobiz',
       'Automatic Traffic Update', 'Bahntwit', 'OxfordBlue-Twitter',
       'autotweet scheduler'

In [26]:
data.loc[data['source'] == 33]

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user


In [27]:
# #Encode every other source as other
# data.loc[(data['source'] != 'web') & (data['source'] != 'mobile'), 'source'] = 'other'
# data.head(10)

data.source.unique()

array(['mobile', 'OLPlusApp', 'Foursquare', 'IFTTT', 'web',
       'Twitter for iPad', 'CareerArc 2.0', nan, 'Instagram',
       'Tweetlogix', 'Trendsmap Alerting', 'Hootsuite Inc.',
       'Foursquare Swarm', 'Tweetbot for iΟS', 'Echofon',
       'dasauge® Stellenmarkt', 'iOS', 'Endomondo', 'manager magazin',
       'TTN ELP traffic', 'PropertyWala.com', '511NY-Tweets',
       '112alarm Lifeliners', 'Echobox Social', 'Cheap Bots, Done Quick!',
       'LA AreaNotice Tweets', 'TweetDeck', 'srvnotif', 'NYRegions',
       '112alarm.net MON', 'twitbot_h', 'SocialFlow', 'Facebook',
       'GroupTweet', 'swonkie', '112app.nl', 'Tuiteo TOP EP (4)',
       'Twitterrific for Mac', '112alarm.net', 'SafeTweet by TweetMyJOBS',
       'Mobolet Tweet', 'iembot', 'TestsForSearch', 'TTN SPO traffic',
       "Edent's Solar Panels", 'Tweetbot for Mac',
       'City of Portland Fire/EMS feed', 'eltiempobiz',
       'Automatic Traffic Update', 'Bahntwit', 'OxfordBlue-Twitter',
       'autotweet scheduler'

In [28]:
#Find out what is in the user column
freqs = data.user.head(1)
for k, v in freqs.items():
    print(f'{k} {v}')

0 {'id': 993959236752338944, 'id_str': '993959236752338944', 'name': 'Zehra Akkurt', 'screen_name': 'ZehraAkkurt20', 'location': 'Berlin, Germany', 'url': None, 'description': 'Dünya gözü ile bakan yüzü;\nGönül gözü ile bakan özü görür...\n\nHz.Mevlana', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 118, 'friends_count': 181, 'listed_count': 0, 'favourites_count': 2028, 'statuses_count': 397, 'created_at': 'Tue May 08 21:02:03 +0000 2018', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': 'tr', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'profile_image_url': 'http://pbs.twimg.com/profile_ima

In [29]:
#Split the user object
tags = data['user'].apply(pd.Series)
tags = tags.rename(columns = lambda x : str(x))
tags

,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,id,id_str,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified,0
0,False,Tue May 08 21:02:03 +0000 2018,True,False,Dünya gözü ile bakan yüzü;\nGönül gözü ile bak...,2028.0,NaN,118.0,NaN,181.0,True,9.939592e+17,993959236752338944,False,tr,0.0,"Berlin, Germany",Zehra Akkurt,NaN,F5F8FA,,,False,https://pbs.twimg.com/profile_banners/99395923...,http://pbs.twimg.com/profile_images/1097208623...,https://pbs.twimg.com/profile_images/109720862...,1DA1F2,C0DEED,DDEEF6,333333,True,False,ZehraAkkurt20,397.0,NaN,none,None,NaN,False,NaN
1,False,Mon May 30 02:58:44 +0000 2011,False,False,2.4 😜 //\nI do not need anyone who did not wan...,2340.0,NaN,270.0,NaN,274.0,True,3.076783e+08,307678258,False,pt,0.0,"Leopoldina, MG",Wemersom 🌈,NaN,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/30767825...,http://pbs.twimg.com/profile_images/1093471586...,https://pbs.twimg.com/profile_images/109347158...,0084B4,C0DEED,DDEEF6,333333,True,False,WmateusLS,2246.0,NaN,none,http://www.facebook.com/wemersom.mateus,NaN,False,NaN
2,False,Sat Aug 22 20:47:16 +0000 2009,False,False,No a girly girl 🙄 💚 my 2 sons 💚💚my grandson 💚 ...,3113.0,NaN,177.0,NaN,368.0,True,6.797930e+07,67979303,False,en,2.0,"Edinburgh, Scotland",Jo Parker,NaN,352726,http://abs.twimg.com/images/themes/theme5/bg.gif,https://abs.twimg.com/images/themes/theme5/bg.gif,False,https://pbs.twimg.com/profile_banners/67979303...,http://pbs.twimg.com/profile_images/1040487813...,https://pbs.twimg.com/profile_images/104048781...,D02B55,829D5E,99CC33,3E4415,True,False,jo31070,1953.0,NaN,none,None,NaN,False,NaN
3,False,Sun Feb 16 14:11:47 +0000 2014,True,False,失去山與林的Kbalan後裔,295.0,NaN,15.0,NaN,87.0,True,2.346919e+09,2346918559,False,zh-TW,0.0,中華民國 台灣 的某個小角落,Jonny.U.Inaba,NaN,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/23469185...,http://pbs.twimg.com/profile_images/7032255906...,https://pbs.twimg.com/profile_images/703225590...,1DA1F2,C0DEED,DDEEF6,333333,True,False,LinXhiang,580.0,NaN,none,None,NaN,False,NaN
4,False,Thu Feb 05 10:09:36 +0000 2009,False,False,🇿🇦RECORDING ARTIST/SONGWRITER | Bookings: Sphu...,18572.0,NaN,6496.0,NaN,1794.0,True,2.013365e+07,20133650,False,en,37.0,South Africa,S P H U M,NaN,FF6699,http://abs.twimg.com/images/themes/theme11/bg.gif,https://abs.twimg.com/images/themes/theme11/bg...,True,https://pbs.twimg.com/profile_banners/20133650...,http://pbs.twimg.com/profile_images/1074884306...,https://pbs.twimg.com/profile_images/107488430...,B40B43,FFFFFF,E5507E,362720,True,False,IAmSphum,59595.0,NaN,none,https://itunes.apple.com/za/album/eighth-wonde...,NaN,False,NaN
5,False,Sat Jul 02 19:42:10 +0000 2011,False,False,A human Being!,1665.0,NaN,3351.0,NaN,397.0,True,3.281243e+08,328124255,False,en,8.0,"WorldWide, Dubai",MoO,NaN,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/32812425...,http://pbs.twimg.com/profile_images/1068246093...,https://pbs.twimg.com/profile_images/106824609...,3B94D9,000000,DDEEF6,333333,True,False,mMohamedOsama,21524.0,NaN,regular,None,NaN,False,NaN
6,False,Fri Oct 14 23:12:34 +0000 2011,True,False,None,10047.0,NaN,328.0,NaN,327.0,True,3.910346e+08,391034611,False,pt,9.0,None,Rita Berlofa 🌹,NaN,C0DEED,http://

In [30]:
#Add columns of interest to the dataset from the user object
data['protected'] = tags.protected
data['tweets_liked'] = tags.favourites_count
data['followers'] = tags.followers_count
data['friends'] = tags.friends_count
data['listed_under'] = tags.listed_count
data['geo_enabled'] = tags.geo_enabled
data['user_location'] = tags.location

data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,retweeted_status,source,text,user,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,NaN,0.0,tr,NaN,0.0,0.0,0.0,NaN,mobile,Pencereni aç ve sabahı kokla. Hayat güzel. Sev...,"{'id': 993959236752338944, 'id_str': '99395923...",False,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,mobile,Isso ae,"{'id': 307678258, 'id_str': '307678258', 'name...",False,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,mobile,F*** you #Cancer another life barely begun sto...,"{'id': 67979303, 'id_str': '67979303', 'name':...",False,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,False,NaN,0.0,und,0.0,0.0,0.0,0.0,NaN,mobile,@parfaite_1223 @parfaite_narumi https://t.co/9...,"{'id': 2346918559, 'id_str': '2346918559', 'na...",False,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,False,NaN,0.0,en,NaN,0.0,0.0,0.0,NaN,mobile,@Mbuthuma I have beeeeen following you! Twitte...,"{'id': 20133650, 'id_str': '20133650', 'name':...",False,18572.0,6496.0,1794.0,37.0,True,South Africa
5,False,NaN,0.0,ar,NaN,0.0,0.0,0.0,NaN,mobile,جاكي شان بيصور الفيلم الجديد بتاعه جنبنا وعامل...,"{'id': 328124255, 'id_str': '328124255', 'name...",False,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,False,NaN,0.0,und,NaN,0.0,0.0,0.0,NaN,mobile,@greenhousenyt 👏👏👏👏👏👏,"{'id': 391034611, 'id_str': '391034611', 'name...",False,10047.0,328.0,327.0,9.0,True,None
7,False,NaN,0.0,ja,NaN,0.0,0.0,0.0,NaN,mobile,そろそろキャンプの準備するか。,"{'id': 237612409, 'id_str': '237612409', 'name...",False,486.0,61.0,122.0,7.0,True,None
8,False,NaN,0.0,pt,NaN,0.0,0.0,0.0,NaN,mobile,@Tkn117 oi lindo t amo,"{'id': 52052164, 'id_str': '52052164', 'name':...",False,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,False,NaN,0.0,tl,NaN,0.0,0.0,0.0,NaN,mobile,@RollyRockza Mbavhala,"{'id': 450095256, 'id_str': '450095256', 'name...",False,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [31]:
#Drop unnecessary columns
data.drop(['text','user','retweeted_status','geo_coordinates','lang'], 1, inplace = True)
data.head(10)

,geo_tag,favorite_count,possibly_sensitive,reply_count,retweet_count,retweeted,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0.0,NaN,0.0,0.0,0.0,mobile,False,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,False,0.0,NaN,0.0,0.0,0.0,mobile,False,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,False,0.0,NaN,0.0,0.0,0.0,mobile,False,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,False,0.0,0.0,0.0,0.0,0.0,mobile,False,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,False,0.0,NaN,0.0,0.0,0.0,mobile,False,18572.0,6496.0,1794.0,37.0,True,South Africa
5,False,0.0,NaN,0.0,0.0,0.0,mobile,False,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,False,0.0,NaN,0.0,0.0,0.0,mobile,False,10047.0,328.0,327.0,9.0,True,None
7,False,0.0,NaN,0.0,0.0,0.0,mobile,False,486.0,61.0,122.0,7.0,True,None
8,False,0.0,NaN,0.0,0.0,0.0,mobile,False,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,False,0.0,NaN,0.0,0.0,0.0,mobile,False,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [32]:
#Rename columns for easier reference
data.rename(columns={'favorite_count': 'likes', 'reply_count': 'replies', 'retweet_count': 'retweets'}, inplace=True)

In [33]:
#Some useful functions

#Function to encode values
def encode_col(col):
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes
    return data.head(10)

#Function to replace null entries with any choice
def impute_nulls(col, null_replacement, data_type):
    data[col].fillna(null_replacement, inplace=True)
    data[col] = data[col].astype(data_type)
    return data.head(10)

In [34]:
#DISADVANTAGE OF LIVE STREAMING TWEETS
#The following columns are dropped as the tweets were streamed live and no data will have been obtained from it

data.drop(['likes','replies', 'retweets', 'retweeted'], 1, inplace = True)
data.head(10)

,geo_tag,possibly_sensitive,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,NaN,mobile,False,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,False,NaN,mobile,False,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,False,NaN,mobile,False,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,False,0.0,mobile,False,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,False,NaN,mobile,False,18572.0,6496.0,1794.0,37.0,True,South Africa
5,False,NaN,mobile,False,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,False,NaN,mobile,False,10047.0,328.0,327.0,9.0,True,None
7,False,NaN,mobile,False,486.0,61.0,122.0,7.0,True,None
8,False,NaN,mobile,False,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,False,NaN,mobile,False,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [35]:
#Function call: Replace null values with 0
impute_nulls('possibly_sensitive', 0, 'int64')

,geo_tag,possibly_sensitive,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0,mobile,False,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,False,0,mobile,False,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,False,0,mobile,False,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,False,0,mobile,False,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,False,0,mobile,False,18572.0,6496.0,1794.0,37.0,True,South Africa
5,False,0,mobile,False,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,False,0,mobile,False,10047.0,328.0,327.0,9.0,True,None
7,False,0,mobile,False,486.0,61.0,122.0,7.0,True,None
8,False,0,mobile,False,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,False,0,mobile,False,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [36]:
data.possibly_sensitive.unique()

array([0, 1], dtype=int64)

In [37]:
#Function call: Encode source column
encode_col('source')

,geo_tag,possibly_sensitive,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0,55,False,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,False,0,55,False,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,False,0,55,False,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,False,0,55,False,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,False,0,55,False,18572.0,6496.0,1794.0,37.0,True,South Africa
5,False,0,55,False,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,False,0,55,False,10047.0,328.0,327.0,9.0,True,None
7,False,0,55,False,486.0,61.0,122.0,7.0,True,None
8,False,0,55,False,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,False,0,55,False,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [38]:
data.protected.unique()

array([False, nan], dtype=object)

In [39]:
#The following column is dropped because it only contains False or null values
data.drop(['protected'], 1, inplace = True)
data.head(10)

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0,55,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,False,0,55,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,False,0,55,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,False,0,55,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,False,0,55,18572.0,6496.0,1794.0,37.0,True,South Africa
5,False,0,55,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,False,0,55,10047.0,328.0,327.0,9.0,True,None
7,False,0,55,486.0,61.0,122.0,7.0,True,None
8,False,0,55,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,False,0,55,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [40]:
#Function call: Encode geo_tag column
encode_col('geo_tag')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028.0,118.0,181.0,0.0,True,"Berlin, Germany"
1,0,0,55,2340.0,270.0,274.0,0.0,True,"Leopoldina, MG"
2,0,0,55,3113.0,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,0,0,55,295.0,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,0,0,55,18572.0,6496.0,1794.0,37.0,True,South Africa
5,0,0,55,1665.0,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,0,0,55,10047.0,328.0,327.0,9.0,True,None
7,0,0,55,486.0,61.0,122.0,7.0,True,None
8,0,0,55,26255.0,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,0,0,55,22653.0,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [41]:
#Function call: Replace null values with 0
impute_nulls('tweets_liked', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118.0,181.0,0.0,True,"Berlin, Germany"
1,0,0,55,2340,270.0,274.0,0.0,True,"Leopoldina, MG"
2,0,0,55,3113,177.0,368.0,2.0,True,"Edinburgh, Scotland"
3,0,0,55,295,15.0,87.0,0.0,True,中華民國 台灣 的某個小角落
4,0,0,55,18572,6496.0,1794.0,37.0,True,South Africa
5,0,0,55,1665,3351.0,397.0,8.0,True,"WorldWide, Dubai"
6,0,0,55,10047,328.0,327.0,9.0,True,None
7,0,0,55,486,61.0,122.0,7.0,True,None
8,0,0,55,26255,380.0,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,0,0,55,22653,6952.0,4342.0,3.0,True,"Onverwacht, South Africa"


In [42]:
#Find out arrangement of the tweets_liked values so as to create categories
data.tweets_liked.describe()

count      4147.000000
mean      17003.698818
std       38729.821203
min           0.000000
25%         965.000000
50%        4814.000000
75%       16551.500000
max      794828.000000
Name: tweets_liked, dtype: float64

In [43]:
#Categorize tweets_liked
# data.tweets_liked = pd.cut(data['tweets_liked'], bins=[0, 158965, 317930, 476895, 635860, 794828], include_lowest=True, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# data.head(10)

In [44]:
#Encode the column
#encode_col('tweets_liked')

In [45]:
impute_nulls('followers', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118,181.0,0.0,True,"Berlin, Germany"
1,0,0,55,2340,270,274.0,0.0,True,"Leopoldina, MG"
2,0,0,55,3113,177,368.0,2.0,True,"Edinburgh, Scotland"
3,0,0,55,295,15,87.0,0.0,True,中華民國 台灣 的某個小角落
4,0,0,55,18572,6496,1794.0,37.0,True,South Africa
5,0,0,55,1665,3351,397.0,8.0,True,"WorldWide, Dubai"
6,0,0,55,10047,328,327.0,9.0,True,None
7,0,0,55,486,61,122.0,7.0,True,None
8,0,0,55,26255,380,274.0,12.0,True,"Rio de Janeiro, Brasil"
9,0,0,55,22653,6952,4342.0,3.0,True,"Onverwacht, South Africa"


In [46]:
#Find out arrangement of the followers values so as to create categories
data.followers.describe()

count    4.147000e+03
mean     4.549913e+03
std      5.722776e+04
min      0.000000e+00
25%      1.735000e+02
50%      4.990000e+02
75%      1.492000e+03
max      2.848613e+06
Name: followers, dtype: float64

In [47]:
# #Categorize followers
# data.followers = pd.cut(data['followers'], bins=[0, 1000, 10000, 100000, 1000000, 2900000], include_lowest=True, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# data.head(10)

In [48]:
# #Encode the column
# encode_col('followers')

In [49]:
impute_nulls('friends', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118,181,0.0,True,"Berlin, Germany"
1,0,0,55,2340,270,274,0.0,True,"Leopoldina, MG"
2,0,0,55,3113,177,368,2.0,True,"Edinburgh, Scotland"
3,0,0,55,295,15,87,0.0,True,中華民國 台灣 的某個小角落
4,0,0,55,18572,6496,1794,37.0,True,South Africa
5,0,0,55,1665,3351,397,8.0,True,"WorldWide, Dubai"
6,0,0,55,10047,328,327,9.0,True,None
7,0,0,55,486,61,122,7.0,True,None
8,0,0,55,26255,380,274,12.0,True,"Rio de Janeiro, Brasil"
9,0,0,55,22653,6952,4342,3.0,True,"Onverwacht, South Africa"


In [50]:
#Find out arrangement of the friends values so as to create categories
data.friends.describe()

count      4147.000000
mean       1389.829998
std        7326.465694
min           0.000000
25%         201.000000
50%         487.000000
75%        1176.000000
max      357435.000000
Name: friends, dtype: float64

In [51]:
#Categorize friends
# data.friends = pd.cut(data['friends'], bins=[0, 71487, 142974, 214461, 285948, 357435], include_lowest=True, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# data.head(10)

In [52]:
#Encode the column
# encode_col('friends')

In [53]:
impute_nulls('listed_under', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118,181,0,True,"Berlin, Germany"
1,0,0,55,2340,270,274,0,True,"Leopoldina, MG"
2,0,0,55,3113,177,368,2,True,"Edinburgh, Scotland"
3,0,0,55,295,15,87,0,True,中華民國 台灣 的某個小角落
4,0,0,55,18572,6496,1794,37,True,South Africa
5,0,0,55,1665,3351,397,8,True,"WorldWide, Dubai"
6,0,0,55,10047,328,327,9,True,None
7,0,0,55,486,61,122,7,True,None
8,0,0,55,26255,380,274,12,True,"Rio de Janeiro, Brasil"
9,0,0,55,22653,6952,4342,3,True,"Onverwacht, South Africa"


In [54]:
data.listed_under.describe()

count     4147.000000
mean        42.892935
std        564.215345
min          0.000000
25%          0.000000
50%          3.000000
75%         14.000000
max      32220.000000
Name: listed_under, dtype: float64

In [55]:
#Categorize listed_under
# data.listed_under = pd.cut(data['listed_under'], bins=[0, 3, 14, 32220], include_lowest=True, labels=['low', 'medium', 'high'])
# data.head(10)

In [56]:
# encode_col('listed_under')

In [57]:
encode_col('geo_enabled')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118,181,0,1,"Berlin, Germany"
1,0,0,55,2340,270,274,0,1,"Leopoldina, MG"
2,0,0,55,3113,177,368,2,1,"Edinburgh, Scotland"
3,0,0,55,295,15,87,0,1,中華民國 台灣 的某個小角落
4,0,0,55,18572,6496,1794,37,1,South Africa
5,0,0,55,1665,3351,397,8,1,"WorldWide, Dubai"
6,0,0,55,10047,328,327,9,1,None
7,0,0,55,486,61,122,7,1,None
8,0,0,55,26255,380,274,12,1,"Rio de Janeiro, Brasil"
9,0,0,55,22653,6952,4342,3,1,"Onverwacht, South Africa"


In [58]:
data['user_location'].fillna('False', inplace=True)
data.loc[data['user_location'] != 'False', 'user_location'] = 'True'
data.head(10)

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118,181,0,1,True
1,0,0,55,2340,270,274,0,1,True
2,0,0,55,3113,177,368,2,1,True
3,0,0,55,295,15,87,0,1,True
4,0,0,55,18572,6496,1794,37,1,True
5,0,0,55,1665,3351,397,8,1,True
6,0,0,55,10047,328,327,9,1,False
7,0,0,55,486,61,122,7,1,False
8,0,0,55,26255,380,274,12,1,True
9,0,0,55,22653,6952,4342,3,1,True


In [59]:
encode_col('user_location')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,55,2028,118,181,0,1,1
1,0,0,55,2340,270,274,0,1,1
2,0,0,55,3113,177,368,2,1,1
3,0,0,55,295,15,87,0,1,1
4,0,0,55,18572,6496,1794,37,1,1
5,0,0,55,1665,3351,397,8,1,1
6,0,0,55,10047,328,327,9,1,0
7,0,0,55,486,61,122,7,1,0
8,0,0,55,26255,380,274,12,1,1
9,0,0,55,22653,6952,4342,3,1,1


In [60]:
#Bring in the model
X = np.array(data.drop(['geo_tag'], 1).astype(float))
y = np.array(data['geo_tag'])
 
#Scale the data to remove influence of outliers
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=2, max_iter=600)
kmeans.fit(X)

correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(int))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X)*100)

24.330841572220884


In [61]:
#Apply individual columns to the model to see effect
testing_data = data.copy()
y = np.array(testing_data['geo_tag'])
testing_data.drop(['geo_tag'], 1, inplace = True)

#Print individual accuracies
for j in range(len(testing_data.columns)):   
    X = np.array(testing_data[testing_data.columns[j]].astype(float))

    #Scale the data to remove influence of outliers
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X.reshape(-1, 1))

    kmeans = KMeans(n_clusters=2, max_iter=600)
    kmeans.fit(X.reshape(-1, 1))

    correct = 0
    for i in range(len(X)):
        predict_me = np.array(X[i].astype(int))
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1

    print(testing_data.columns[j] , '=>' ,correct/len(X)*100)

possibly_sensitive => 91.1984567156981
source => 15.746322642874366
tweets_liked => 91.39136725343621
followers => 91.39136725343621
friends => 91.39136725343621
listed_under => 91.39136725343621
geo_enabled => 83.77140101278032
user_location => 75.66915842777911
